In [1]:
import pandas as pd
import matplotlib
matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt
import csv

In [103]:
# Create a dictionary that maps the run# to host_density { run#: host_density }

paramfile = "/media/hill/DATA-LINUX/abm-data/host-density/testparams2hab5"
paramd = {}
with open(paramfile, 'r') as file:
    for line in file:
        result = line.replace("\t",",").replace('\n','').split(',')

        # attempt to add the density value to run, if it doesn't exist, create it
        try:
            paramd[int(result[0])].append((float(result[4])))
        except KeyError:
            paramd[int(result[0])] = (float(result[4]))  # { run#: host_density }

for k, v in paramd.items():
    print(k,v)

1 0.01
2 0.03
3 0.05
4 0.03
5 0.05


In [107]:
# Trying to eliminate some steps here...
# step 2
colnames = ['run', 'tick', 'lifestate', 'total_ixodes']
csv_file = "/media/hill/DATA-LINUX/abm-data/host-density/testdf1hab"
df = pd.read_csv(csv_file, names=colnames, header=None, error_bad_lines=False)
n_ticks = df.groupby(['run'], as_index=False)
# n_ticks = n_ticks.agg(total_ixodes=pd.NamedAgg(column='name', aggfunc='nunique'))
n_ticks = n_ticks.agg({'total_ixodes': 'nunique'})
n_ticks.dtypes




run             int64
total_ixodes    int64
dtype: object

In [108]:
# step 3
# Iterate through param dictionary and add the host density value to the associated run
for key, value in paramd.items():
    print(key)
    n_ticks.loc[n_ticks['run']==key, 'host_density' ] = float(value)
n_ticks.head()


1
2
3
4
5


,run,total_ixodes,host_density
0,1,11,0.01
1,2,10,0.03
2,3,10,0.05
3,4,17,0.03
4,5,22,0.05


In [109]:
# step 4 make the final df
n_ticks = n_ticks.groupby('host_density', as_index=False)
n_ticks = n_ticks.agg({'total_ixodes': 'mean'})
n_ticks.head()

,host_density,total_ixodes
0,0.01,11.0
1,0.03,13.5
2,0.05,16.0


In [61]:
# Create a dictionary that maps the run# to total ixode { run#: cumulative_ixode }
ixode_count_dict = {}
colnames = ['run', 'tick', 'lifestate', 'name']
csv_file = "/media/hill/DATA-LINUX/abm-data/host-density/testdf1hab"
df = pd.read_csv(csv_file, names=colnames, header=None, error_bad_lines=False)
for run in df.groupby('run'):
    current_df = run[1]
    ixode_count_dict[run[0]] = len(current_df['name'].unique())


In [97]:
# Creating a data frame from the cumulative_ixodes dict generated above
df_final = pd.DataFrame(ixode_count_dict.items(), columns=['run', 'total_ixodes'])
df_final['host_density'] = 0
df_final.dtypes

NameError: name 'ixode_count_dict' is not defined

In [100]:
# Iterate through param dictionary and add the host density value to the associated run
for key, value in paramd.items():
    for i in range(len(paramd)):
        # print(type(value))
        df_final.loc[df_final['run']==key, 'host_density'] = value

df_final.dtypes

NameError: name 'df_final' is not defined

In [58]:
# Now we have a df with | 'run' | 'total_ixode' | 'host_density' |
# Create a dictionary with { host_density: agg_ixodes }
agg_ixodes_dict = {}
for density in df_final.groupby('host_density'):
    tmp_df = density[1] # density is a tuple, so density[1] is the df we want
    print(density[0])
    agg_ixodes_dict[density[0]] = tmp_df['total_ixodes'].agg('mean')


# in test file - should get 1880.75 for 0.4, 7898.5 for 0.7, and 24917 for 1.0


0.7


In [52]:
# Create a dataframe from the agg_ixodes_dict that we can then plot
df_agg_final = pd.DataFrame(agg_ixodes_dict.items(), columns=['host_density', 'agg_ixodes'])
df_agg_final.head()

,host_density,agg_ixodes
0,0.7,13.8


In [9]:
matplotlib.use('Qt5Agg')
plt.plot(df_agg_final['host_density'], df_agg_final['agg_ixodes'])
plt.show(block=True)

In [33]:
with open(paramfile, 'r') as file:
    reader = csv.reader(file)
    param_str = {rows[0]: "Lifestage: {}\nStarting Ixodes: {}\nHabitat Suitability: {}".format(
        rows[1], rows[2], rows[3]) for rows in reader}
    print(param_str)

{'1\ttick_lifestage\tadult': 'Lifestage: large_host_density\t0.7\nStarting Ixodes: tick_count\t10\nHabitat Suitability: habitat_suitability\t0.01', '2\ttick_lifestage\tadult': 'Lifestage: large_host_density\t0.7\nStarting Ixodes: tick_count\t10\nHabitat Suitability: habitat_suitability\t0.03', '3\ttick_lifestage\tadult': 'Lifestage: large_host_density\t0.7\nStarting Ixodes: tick_count\t10\nHabitat Suitability: habitat_suitability\t0.05', '4\ttick_lifestage\tadult': 'Lifestage: large_host_density\t0.7\nStarting Ixodes: tick_count\t10\nHabitat Suitability: habitat_suitability\t0.07', '5\ttick_lifestage\tadult': 'Lifestage: large_host_density\t0.7\nStarting Ixodes: tick_count\t10\nHabitat Suitability: habitat_suitability\t0.1'}


In [40]:
fig, ax = plt.subplots()
ax.set_title("Test Agg Host Density")
ax.set_ylabel("Total Ixodes")
ax.set_xlabel("Host Density")
props = dict(facecolor='wheat', alpha=0.5)
ax.plot(df_agg_final['host_density'], df_agg_final['agg_ixodes'])
plt.figtext(0.5,0.5, 'Lifestage: adult\nStarting Ixodes: 10\nHabitat Suitability: 0.05', bbox=props)
plt.show(block=True)

In [49]:
df2 = df_agg_final.copy()
ax=df2.plot()
df_agg_final.plot(ax=ax)
plt.show(block=True)

